In [6]:
from datetime import datetime
import sqlite3
import time
import sys
from bs4 import BeautifulSoup
import requests
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import schedule

try:
    page = requests.get('https://www.autocentrum.pl/paliwa/ceny-paliw/slaskie/') #ceny
    page2 = requests.get('https://www.e-petrol.pl/notowania/rynek-krajowy/hurt') #hurt_ceny
    page3 = requests.get('https://www.bankier.pl/inwestowanie/profile/quote.html?symbol=ROPA')
except Exception as e:
    error_type, error_obj, error_info = sys.exc_info()
    print ('error')
    print (error_type, 'line', error_info.tb_lineo)
    
time.sleep(1)

soup = BeautifulSoup(page.text,'html.parser')
soup2 = BeautifulSoup(page2.text,'html.parser')
soup3 = BeautifulSoup(page3.text,'html.parser')

#cena w zł na śląsku
benzyna_95 = soup.find('div',attrs={'class':'price'})
diesiel = soup.find('a',attrs={'href':'/paliwa/ceny-paliw/slaskie/on/'})

#hurtowe ceny benzyny95
hurt_95 = soup2.find("div",  text=" Pb 95")
hurt_95_cena = soup2.find('div',attrs={'class':'col pb95'})

#hurtowe ceny diesiel
hurt_ON = soup2.find("div",  text=" ON")
hurt_ON_cena = soup2.find('div',attrs={'class':'col one'})

#ceny ropy
Ropa = soup3.find('div',attrs={'class':'profilLast'})


In [7]:
#CENY PALIWA I CENY HURTOWE
print("Benzyna95\n",benzyna_95.text)
print("Diesiel\n",diesiel.text)
print("HurtoweCeny\n",  hurt_95.text, hurt_95_cena.text,  "zł/m3")

print(hurt_ON.text,"-", hurt_ON_cena.text, "zł/m3")




Benzyna95
 
                                                    7,15
                            zł

Diesiel
 
ON

                                                    7,48
                            zł


HurtoweCeny
  Pb 95 6497 zł/m3
 ON - 6895 zł/m3


In [8]:
print(Ropa.text)

103,98 USD/baryÅka


In [9]:
#BAZA DANYCH

conn = sqlite3.connect('cenypaliwo.db')

c = conn.cursor()

#c.execute(' ' ' CREATE TABLE paliwodiesel (rodzaj TEXT, cena TEXT, data TEXT) ' ' ')

#TABELA DIESEL
rodzaj = 'Diesel'
cena = diesiel.text
data = datetime.now()
c.execute(' ' ' INSERT INTO  paliwodiesel VALUES (?, ?,?) ' ' ',  (rodzaj, cena, data))

#TABELA BENZYNA
rodzaj2 = 'Benzyna95'
cena2 = benzyna_95.text
data2 = datetime.now()
c.execute(' ' ' INSERT INTO  paliwo95 VALUES (?, ?,?) ' ' ',  (rodzaj2, cena2, data2))






#c.execute(' ' ' DELETE FROM paliwo95 WHERE rodzaj=Diesel ;' ' ')
conn.commit()

In [10]:


# send a request to fetch HTML of the page
response = requests.get('https://www.autocentrum.pl/paliwa/ceny-paliw/slaskie/')

# create the soup object
soup = BeautifulSoup(response.text, 'html.parser')

# change the encoding to utf-8
soup.encode('utf-8')


def check_price():
  benzyna_95 = "benzyna"
  price = soup.find('div',attrs={'class':'price'})
  print(price)

  #converting the string amount to float
  converted_price = price
  print(converted_price)
  if(converted_price >0):
    send_mail()

  #using strip to remove extra spaces in the title
  print(benzyna_95.strip())





# function that sends an email if the prices fell down
def send_mail():
  server = smtplib.SMTP('smtp.gmail.com', 587)
  server.ehlo()
  server.starttls()
  server.ehlo()

  server.login('faptribute987@gmail.com', 'tyhuju03')


  body = "Cena paliwa spadła na 6,48 https://www.autocentrum.pl/paliwa/ceny-paliw/slaskie/ "

  
  msg = MIMEMultipart()
  msg['From'] = 'Paliwo'
  msg['To'] = 'faptribute987@gmail.com'
  msg['Subject']= "Price Fell Down"
  msg.attach(MIMEText(body, 'plain'))
  
  server.sendmail(
    'sender@gmail.com',
    'faptribute987@gmail.com',
    msg.as_string()
  )
  #print a message to check if the email has been sent
  print('Hey Email has been sent')
  # quit the server
  server.quit()

#loop that allows the program to regularly check for prices

#schedule.every().day.at("14:02").do(check_price)

#while (True):
    #schedule.run_pending()
    #time.sleep(1)

